In [362]:
import pandas as pd
import numpy as np

In [363]:
space_units_df = pd.read_csv('../Data/space_units_df.csv', index_col='Name')
ground_units_df= pd.read_csv('../Data/ground_units_df.csv', index_col='Name')

In [364]:
ground_units_df

,Cost,Combat,Combat_Value,Ability,Unit Type
Name,,,,,
Infantry,1 (x2),8^,8,NaN,Infantry
Infantry II,1 (x2),7,7,"After this unit is destroyed, roll 1 die. If t...",Infantry
Spec Ops I,1 (x2),7^,7,NaN,Infantry
Spec Ops II,1 (x2),6,6,"After this unit is destroyed, roll 1 die. If t...",Infantry
Letani Warrior I,1 (x2),8^,8,Production 1,Infantry
Letani Warrior II,1 (x2),7,7,"After this unit is destroyed, roll 1 die. If t...",Infantry
Crimson Legionnaire I,1 (x2),8^,8,"After this unit is destroyed, gain 1 commodity...",Infantry
Crimson Legionnaire II,1 (x2),7,7,"After this unit is destroyed, gain 1 commodity...",Infantry
Letani Behemoth,2,6,6,DEPLOY: When you use MITOSIS faction ability y...,Mechs


In [365]:
# Set Ships

faction_A_units = {'Carrier': 1}
faction_B_units = {'Dreadnought': 1}

In [366]:
# Find Combat Values

def get_ship_stats(faction_units):
    faction_ship_stats = []
    for unit in faction_units:
        number_units = faction_units[unit]
        for i in range(number_units):
            name = unit
            unit_combat_value = int(space_units_df.loc[unit]['Combat_Value'])
            ship_stats = {'Name': name, 'Unit_Combat_Value': unit_combat_value}
            faction_ship_stats.append(ship_stats)
    return faction_ship_stats

faction_A_ships = get_ship_stats(faction_A_units)
faction_B_ships = get_ship_stats(faction_B_units)

In [367]:
# Determine Hits

def get_ship_hits(faction_ships):
    hits = 0
    for ship in faction_ships:
        roll = np.random.randint(1, 11)
        if roll >= ship['Unit_Combat_Value']:
            hits += 1
    return hits

faction_A_hits = get_ship_hits(faction_A_ships)
faction_B_hits = get_ship_hits(faction_B_ships)

In [368]:
# Assign Hits

for i in range(faction_A_hits):
    faction_B_ships.remove(faction_B_ships[0])

for i in range(faction_B_hits):
    faction_A_ships.remove(faction_A_ships[0])



In [369]:
faction_A_ships

[{'Name': 'Carrier', 'Unit_Combat_Value': 9}]